# Disability Insurance Calculator

This notebook calculates disability insurance coverage needs by comparing baseline income projections (with job income) against disability scenarios (without job income). It accounts for:

- Lost job income until age 65
- Reduced Social Security benefits (including post-65 reductions)
- Reduced pension benefits (including post-65 reductions)
- Existing employer-provided disability coverage (after taxes)
- Remaining coverage gap in standard disability insurance format (% of income for x years)

**Usage**: Load your user configuration file (YAML) with income profiles, Social Security settings, pension settings, and existing disability coverage details.

In [ ]:
"""Disability Insurance Calculator

This notebook calculates disability insurance coverage needs by comparing
baseline income projections against disability scenarios. It uses SimulationEngine
to calculate income streams and Social Security benefits, avoiding direct use
of individual controllers due to tight coupling.

Note: This notebook must be run from the workspace root directory, or the
working directory must be changed to the workspace root before importing.
"""

import sys
import os
from pathlib import Path

# Determine workspace root directory
# This is needed because the notebook is in a subdirectory
notebook_dir = Path.cwd()
workspace_root = notebook_dir.parent if notebook_dir.name == "standalone_tools" else notebook_dir

# Add workspace root to path FIRST, before any imports
if str(workspace_root) not in sys.path:
    sys.path.insert(0, str(workspace_root))

# Change working directory to workspace root so relative paths work
# This is critical - must be done before any app module imports
os.chdir(workspace_root)

# Set environment variable to skip Flask app initialization
# This prevents circular import issues when importing app modules
os.environ["SKIP_FLASK_INIT"] = "1"

# Import standard library and third-party packages first
from typing import Optional, Dict, Tuple
import yaml
import pandas as pd
import numpy as np

# Import app modules - Flask app initialization is skipped via SKIP_FLASK_INIT
from app.data.constants import CONFIG_PATH, TODAY_YR_QT
from app.models.config import User, get_config
from app.models.simulator import SimulationEngine, Results
from app.util import interval_yield

## Configuration Loading

Load user configuration from YAML file and extract disability coverage details.

In [ ]:
def load_config(config_path: Path = CONFIG_PATH) -> Tuple[User, Dict]:
    """Load user config from YAML file and extract disability coverage fields.
    
    Args:
        config_path: Path to YAML config file
        
    Returns:
        Tuple of (User config object, disability coverage dict)
        Disability coverage dict contains:
        - user_disability_coverage: Dict with 'percentage' (float) and 'duration_years' (int)
        - partner_disability_coverage: Optional dict with 'percentage' and 'duration_years'
    """
    user_config = get_config(config_path)
    
    # Read raw YAML to extract disability coverage (not in User model yet)
    with open(config_path, "r", encoding="utf-8") as f:
        yaml_content = yaml.safe_load(f)
    
    disability_coverage = {
        "user_disability_coverage": yaml_content.get("user_disability_coverage", {
            "percentage": 0.0,
            "duration_years": 0
        }),
        "partner_disability_coverage": yaml_content.get("partner_disability_coverage", {
            "percentage": 0.0,
            "duration_years": 0
        })
    }
    
    return user_config, disability_coverage

# Load configuration
# Config file is in the workspace root (current working directory after os.chdir)
config_path = Path("config.yml")
if not config_path.exists():
    # Fallback to default CONFIG_PATH if not found
    config_path = CONFIG_PATH
user_config, disability_coverage = load_config(config_path)

print(f"Config loaded from: {config_path}")
print(f"User age: {user_config.age}")
print(f"User disability coverage: {disability_coverage['user_disability_coverage']}")
if user_config.partner:
    print(f"Partner age: {user_config.partner.age}")
    print(f"Partner disability coverage: {disability_coverage['partner_disability_coverage']}")

In [ ]:
# Benefit cutoff age configuration
# This is configurable - user can set a different age if needed
# Default value is 65 (typical for long-term disability insurance)
BENEFIT_CUTOFF_AGE = 65  # Change this value if you need a different Benefit cutoff age

print(f"Benefit cutoff age: {BENEFIT_CUTOFF_AGE}")


## Configuration Validation

Validate config and handle edge cases: no income profiles, already retired.

In [ ]:
def validate_config(user_config: User) -> Tuple[bool, Optional[str]]:
    """Validate config and check for edge cases.
    
    Args:
        user_config: User configuration object
        
    Returns:
        Tuple of (is_valid, error_message)
        If is_valid is False, error_message contains the reason
    """
    # Check if user or partner have income profiles
    user_has_income = user_config.income_profiles and len(user_config.income_profiles) > 0
    partner_has_income = (
        user_config.partner 
        and user_config.partner.income_profiles 
        and len(user_config.partner.income_profiles) > 0
    )
    
    if not user_has_income and not partner_has_income:
        return False, "No disability insurance needed - no future income to protect"
    
    # Check if already retired (all income profiles have zero income)
    all_zero_income = True
    if user_has_income and user_config.income_profiles:
        for profile in user_config.income_profiles:
            if profile.starting_income > 0:
                all_zero_income = False
                break
    if partner_has_income and all_zero_income and user_config.partner and user_config.partner.income_profiles:
        for profile in user_config.partner.income_profiles:
            if profile.starting_income > 0:
                all_zero_income = False
                break
    
    if all_zero_income:
        return False, "No disability insurance needed - already retired"
    
    return True, None

# Validate configuration
is_valid, error_message = validate_config(user_config)
if not is_valid:
    print(f"ERROR: {error_message}")
    raise SystemExit(error_message)

print("Configuration validated successfully")

## Helper Functions

Helper functions for config modification, income extraction, and age calculations.

In [ ]:
class RealFinancialData():
    """Container for inflation-adjusted (real) financial data from simulation results.
    
    This class extracts income streams and tax data from a simulation results DataFrame,
    converts nominal quarterly values to real (inflation-adjusted) values, and provides
    convenient properties for calculating post-tax income and lifetime totals.
    
    Args:
        df: DataFrame containing simulation results with columns:
            - "Job Income": Quarterly nominal job income
            - "SS User": Quarterly nominal Social Security benefits for user
            - "SS Partner": Quarterly nominal Social Security benefits for partner
            - "Pension": Quarterly nominal pension income
            - "Income Taxes": Quarterly nominal income taxes (negative values)
            - "Medicare Taxes": Quarterly nominal Medicare taxes (negative values)
            - "Date": Date index for each quarter
            - "Inflation": Cumulative inflation factor for each quarter
    
    Attributes:
        dates: Date index from the input DataFrame
        job_real_q: Real (inflation-adjusted) quarterly job income
        ss_user_real_q: Real quarterly Social Security benefits for user
        ss_partner_real_q: Real quarterly Social Security benefits for partner
        pension_real_q: Real quarterly pension income
        income_taxes_real_q: Real quarterly income taxes (negative values)
        medicare_taxes_real_q: Real quarterly Medicare taxes (negative values)
    
    Properties:
        post_tax_income: Real quarterly post-tax income (income + taxes)
        post_tax_total_lifetime: Sum of all real post-tax income over lifetime
        pre_tax_job_total_lifetime: Sum of all real job income over lifetime
        pre_tax_ss_total_lifetime: Sum of all real Social Security benefits over lifetime
        pre_tax_pension_total_lifetime: Sum of all real pension income over lifetime
        pre_tax_total_lifetime: Sum of all real pre-tax income over lifetime
    """
    def __init__(self, df: pd.DataFrame):
        # Extract income streams and taxes (quarterly nominal)
        job_q = df["Job Income"]
        ss_user_q = df["SS User"]
        ss_partner_q = df["SS Partner"]
        pension_q = df["Pension"]
        income_taxes_q = df["Income Taxes"]
        medicare_taxes_q = df["Medicare Taxes"]
        self.dates = df["Date"]
        inflation = df["Inflation"]

        # Convert to real values (adjust for inflation)
        self.job_real_q = job_q / inflation
        self.ss_user_real_q = ss_user_q / inflation
        self.ss_partner_real_q = ss_partner_q / inflation
        self.pension_real_q = pension_q / inflation
        self.income_taxes_real_q = income_taxes_q / inflation
        self.medicare_taxes_real_q = medicare_taxes_q / inflation
    
    @property
    def post_tax_income(self) -> pd.Series:
        income = (self.job_real_q + 
            self.ss_user_real_q + 
            self.ss_partner_real_q + 
            self.pension_real_q)
        taxes = (self.income_taxes_real_q + self.medicare_taxes_real_q)
        return income + taxes # Taxes are already negative
    
    @property
    def post_tax_total_lifetime(self):
        return self.post_tax_income.sum()
    
    @property
    def pre_tax_job_total_lifetime(self):
        return self.job_real_q.sum()
    
    @property
    def pre_tax_ss_total_lifetime(self):
        return (self.ss_user_real_q + self.ss_partner_real_q).sum()
    
    @property
    def pre_tax_pension_total_lifetime(self):
        return self.pension_real_q.sum()
    
    @property
    def pre_tax_total_lifetime(self):
        return self.pre_tax_job_total_lifetime + self.pre_tax_ss_total_lifetime + self.pre_tax_pension_total_lifetime

def set_fixed_inflation(engine: SimulationEngine, inflation_rate: float) -> None:
    """Set fixed inflation rate for all trials in the simulation engine.
    
    This function overrides the simulated cumulative inflation path with a
    deterministic one based on the specified annual inflation rate, ensuring
    stable real income levels for downstream processing.
    
    Args:
        engine: SimulationEngine instance
        inflation_rate: Annual inflation rate as a decimal (e.g., 0.02 for 2%)
    """
    user_config = engine._user_config
    interval_inflation_yield = interval_yield(1 + inflation_rate)
    intervals = user_config.intervals_per_trial
    # Cumulative inflation series starting at 1.0
    cumulative = np.array([interval_inflation_yield ** i for i in range(intervals)], dtype=float)
    # `_economic_sim_data.inflation` has shape (trial_qty, intervals_per_trial)
    engine._economic_sim_data.inflation[:, :] = cumulative

def get_current_annual_income(user_config: User, is_partner: bool = False) -> float:
    """Extract current annual income (first profile with income > $0).
    
    Args:
        user_config: User configuration
        is_partner: If True, use partner income profiles
        
    Returns:
        Current annual income (starting_income is already annualized)
    """
    profiles = user_config.partner.income_profiles if (is_partner and user_config.partner) else user_config.income_profiles
    
    if not profiles:
        return 0.0
    
    # Find first profile with income > $0
    for profile in profiles:
        if profile.starting_income > 0:
            return profile.starting_income
    
    return 0.0

def calculate_years_until_benefit_cutoff_age(age: int, benefit_cutoff_age: int = BENEFIT_CUTOFF_AGE) -> float:
    """Calculate years until Benefit cutoff age.
    
    Args:
        age: Current age
        benefit_cutoff_age: Benefit cutoff age (defaults to BENEFIT_CUTOFF_AGE)
        
    Returns:
        Years until Benefit cutoff age
    """
    return max(0, benefit_cutoff_age - age)

# Test helper functions
print(f"User current annual income: ${get_current_annual_income(user_config, is_partner=False):,.2f}")
print(f"Years until Benefit cutoff age (user): {calculate_years_until_benefit_cutoff_age(user_config.age):.2f}")
if user_config.partner and user_config.partner.age is not None:
    print(f"Partner current annual income: ${get_current_annual_income(user_config, is_partner=True):,.2f}")
    print(f"Years until Benefit cutoff age (partner): {calculate_years_until_benefit_cutoff_age(user_config.partner.age):.2f}")

## User Disability Insurance Calculation

Calculate disability insurance needs for the user.

### Baseline Scenario

Run baseline simulation with original config to get expected income streams.

In [ ]:
# Run baseline scenario (original config)
# Create engine using config_path (following tpaw_planner.ipynb pattern)

try:
    engine = SimulationEngine(config_path=config_path, trial_qty=1)
    set_fixed_inflation(engine, 0.02)  # Set 2% annual inflation for deterministic analysis
    engine.gen_all_trials()
    
    baseline_results = engine.results
    baseline_dfs = baseline_results.as_dataframes()
    assert len(baseline_dfs) == 1, "Expected exactly one trial DataFrame"
    baseline_df = baseline_dfs[0]
    baseline = RealFinancialData(baseline_df)
except Exception as e:
    print(f"ERROR: Failed to run baseline scenario: {e}")
    print("Please check your configuration file and ensure all required fields are present.")
    raise


# Sanity check: Display baseline income totals
benefit_cutoff_age_date = BENEFIT_CUTOFF_AGE - user_config.age + TODAY_YR_QT
mask_until_benefit_cutoff_age = baseline.dates <= benefit_cutoff_age_date

print("=== Baseline Scenario Income Totals (Pre-Tax) ===")
print(f"Job Income (lifetime): ${baseline.pre_tax_job_total_lifetime:,.2f}")
print(f"Social Security (lifetime): ${baseline.pre_tax_ss_total_lifetime:,.2f}")
print(f"Pension (lifetime): ${baseline.pre_tax_pension_total_lifetime:,.2f}")
print(f"Total Pre-Tax Income: ${baseline.pre_tax_total_lifetime:,.2f}")
print()
print("=== Baseline Scenario Post-Tax Income Totals ===")
print(f"Post-Tax Income (lifetime): ${baseline.post_tax_total_lifetime:,.2f}")


### Disability Scenario

Run disability simulation with user job income set to zero.

In [ ]:
# Modify engine config to zero out user job income for disability scenario
# Following tpaw_planner.ipynb pattern of modifying engine data directly
if engine._user_config.income_profiles:
    engine._user_config.income_profiles = []

# Clear results and run disability scenario with modified config
try:
    engine.results = Results()  # Clear previous results
    engine.gen_all_trials()
    
    disability_results = engine.results
    disability_dfs = disability_results.as_dataframes()
    assert len(disability_dfs) == 1, "Expected exactly one trial DataFrame"
    disability_df = disability_dfs[0]
    disability = RealFinancialData(disability_df)
except Exception as e:
    print(f"ERROR: Failed to run disability scenario: {e}")
    print("Please check your configuration file and ensure all required fields are present.")
    raise

# Sanity check: Display disability income totals
print("=== Disability Scenario Income Totals (Pre-Tax) ===")
print(f"Job Income (lifetime): ${disability.pre_tax_job_total_lifetime:,.2f}")
print(f"Social Security (lifetime): ${disability.pre_tax_ss_total_lifetime:,.2f}")
print(f"Pension (lifetime): ${disability.pre_tax_pension_total_lifetime:,.2f}")
print(f"Total Pre-Tax Income: ${disability.pre_tax_total_lifetime:,.2f}")
print()
print("=== Disability Scenario Post-Tax Income Totals ===")
print(f"Post-Tax Income (lifetime): ${disability.post_tax_total_lifetime:,.2f}")


### Income Comparison

Calculate income differences between baseline and disability scenarios.

In [ ]:
# Calculate total replacement needs using post-tax income
# Per FR-005 and FR-008: Total replacement needs = Baseline post-tax income - Disability post-tax income
# where post-tax income = (Job Income + Social Security + Pension) - (Income Taxes + Medicare Taxes)
total_replacement_needs = baseline.post_tax_total_lifetime - disability.post_tax_total_lifetime

# Also calculate component breakdown for display (pre-tax for reference)
# SUM of lost job income + reduced SS + reduced pension (per FR-005 and FR-008)
lost_pre_tax_job_income = baseline.pre_tax_job_total_lifetime - disability.pre_tax_job_total_lifetime
reduced_pre_tax_ss = baseline.pre_tax_ss_total_lifetime - disability.pre_tax_ss_total_lifetime
reduced_pre_tax_pension = baseline.pre_tax_pension_total_lifetime - disability.pre_tax_pension_total_lifetime

# Sanity check: Display income differences
print("=== Income Differences (Baseline - Disability) ===")
print("Pre-Tax Components (for reference):")
print(f"Lost Job Income (until Benefit cutoff age): ${lost_pre_tax_job_income:,.2f}")
print(f"Reduced Social Security (lifetime, including post-Benefit cutoff age): ${reduced_pre_tax_ss:,.2f}")
print(f"Reduced Pension (lifetime, including post-Benefit cutoff age): ${reduced_pre_tax_pension:,.2f}")
# Total replacement needs is the SUM of all components (per FR-005 and FR-008)
print()
print("Post-Tax Total Replacement Needs:")
print(f"  Baseline Post-Tax Income (lifetime): ${baseline.post_tax_total_lifetime:,.2f}")
print(f"  Disability Post-Tax Income (lifetime): ${disability.post_tax_total_lifetime:,.2f}")
print(f"  Total Replacement Needs (post-tax): ${total_replacement_needs:,.2f}")


### Existing Coverage Replacement Calculation

Calculate net after-tax benefits from existing employer-provided disability coverage.

In [ ]:
def calculate_post_tax_existing_coverage(
    baseline_df: pd.DataFrame,
    coverage_percentage: float,
    coverage_duration_years: float,
) -> Tuple[float, Dict]:
    """Calculate existing coverage replacement (net after taxes).
    
    Args:
        baseline_df: Baseline scenario results DataFrame
        coverage_percentage: Coverage percentage (can exceed 100%)
        coverage_duration_years: Coverage duration in years
        
    Returns:
        Tuple of (total_net_replacement, breakdown_dict)
        breakdown_dict contains: gross_benefits, taxes, net_benefits
    """
    # Extract job income and dates from baseline
    job_income_q = baseline_df["Job Income"]
    dates = baseline_df["Date"]
    inflation = baseline_df["Inflation"]
    income_taxes_q = baseline_df["Income Taxes"]
    medicare_taxes_q = baseline_df["Medicare Taxes"]
    
    # Convert to real quarterly
    job_income_real_q = job_income_q / inflation
    
    # Coverage period
    coverage_start = dates.iloc[0]
    coverage_end = min(coverage_start + coverage_duration_years, benefit_cutoff_age_date)
    coverage_mask = (dates >= coverage_start) & (dates <= coverage_end)
    
    # Apply coverage percentage (capped at 100% for calculation)
    coverage_pct_capped = min(coverage_percentage, 100.0) / 100.0
    coverage_benefits_real_q = job_income_real_q[coverage_mask] * coverage_pct_capped
    
    # Calculate taxes on benefits (employer-provided benefits are taxable)
    # Per FR-007: Use average tax rate from baseline scenario for covered intervals
    # Average tax rate = (total income taxes + total Medicare taxes) / total income for coverage period
    total_income_q = job_income_q[coverage_mask]
    total_taxes_q = income_taxes_q[coverage_mask] + medicare_taxes_q[coverage_mask]
    
    # Avoid division by zero
    avg_tax_rate = (-total_taxes_q.sum() / total_income_q.sum()) if total_income_q.sum() > 0 else 0.0
    
    # Calculate taxes on benefits
    taxes_on_benefits_real_q = coverage_benefits_real_q * avg_tax_rate
    
    # Net after-tax benefits
    net_benefits_real_q = coverage_benefits_real_q - taxes_on_benefits_real_q
    
    # Sum
    total_net_replacement = net_benefits_real_q.sum()
    
    gross_benefits = coverage_benefits_real_q.sum()
    taxes = taxes_on_benefits_real_q.sum()
    
    breakdown = {
        "gross_benefits": gross_benefits,
        "taxes": taxes,
        "net_benefits": total_net_replacement
    }
    
    return total_net_replacement, breakdown

# Calculate existing coverage replacement for user
user_coverage = disability_coverage["user_disability_coverage"]
user_coverage_pct = user_coverage.get("percentage", 0.0)
user_coverage_duration = user_coverage.get("duration_years", 0)

if user_coverage_pct > 0 and user_coverage_duration > 0:
    user_existing_coverage, user_coverage_breakdown = calculate_post_tax_existing_coverage(
        baseline_df, user_coverage_pct, user_coverage_duration
    )
    
    print("=== User Existing Coverage Replacement (After Taxes) ===")
    print(f"Coverage: {user_coverage_pct}% for {user_coverage_duration} years")
    print(f"Gross Benefits: ${user_coverage_breakdown['gross_benefits']:,.2f}")
    print(f"Taxes: ${user_coverage_breakdown['taxes']:,.2f}")
    print(f"Net Benefits: ${user_coverage_breakdown['net_benefits']:,.2f}")
else:
    user_existing_coverage = 0.0
    user_coverage_breakdown = {"gross_benefits": 0.0, "taxes": 0.0, "net_benefits": 0.0}
    print("=== User Existing Coverage ===")
    print("No existing coverage")

### Coverage Gap and Benefit Percentage Calculation

Calculate remaining coverage gap and recommended benefit percentage.

In [ ]:
# Calculate coverage gap
coverage_gap = max(0, total_replacement_needs - user_existing_coverage)

# Calculate benefit percentage
# Formula per FR-008: (Coverage gap / Years until Benefit cutoff age) / Current annual income
years_until_benefit_cutoff_age = calculate_years_until_benefit_cutoff_age(user_config.age)
current_annual_income = get_current_annual_income(user_config, is_partner=False)

if years_until_benefit_cutoff_age > 0 and current_annual_income > 0:
    benefit_percentage = (coverage_gap / years_until_benefit_cutoff_age) / current_annual_income * 100
else:
    benefit_percentage = 0.0

print("=== User Coverage Gap Calculation ===")
print(f"Total Replacement Needs: ${total_replacement_needs:,.2f}")
print(f"Existing Coverage Replacement: ${user_existing_coverage:,.2f}")
print(f"Remaining Coverage Gap: ${coverage_gap:,.2f}")
print(f"\nRecommended Coverage:")
print(f"  Benefit Percentage: {benefit_percentage:.1f}%")
print(f"  Duration: {years_until_benefit_cutoff_age:.1f} years (until age {BENEFIT_CUTOFF_AGE})")

### User Results Summary

Structured output for user disability insurance needs.

In [ ]:
print("=" * 70)
print("USER DISABILITY INSURANCE CALCULATION RESULTS")
print("=" * 70)
print()
print(f"1. Total Income Replacement Needed: ${total_replacement_needs:,.2f}")
print(f"   - Lost Job Income (until Benefit cutoff age): ${lost_pre_tax_job_income:,.2f}")
print(f"   - Reduced Social Security (lifetime, including post-Benefit cutoff age): ${reduced_pre_tax_ss:,.2f}")
print(f"   - Reduced Pension (lifetime, including post-Benefit cutoff age): ${reduced_pre_tax_pension:,.2f}")
print()
print(f"2. Existing Coverage Replacement (after taxes): ${user_existing_coverage:,.2f}")
if user_coverage_pct > 0:
    print(f"   - Coverage: {user_coverage_pct}% of income for {user_coverage_duration} years")
    print(f"   - Gross Benefits: ${user_coverage_breakdown['gross_benefits']:,.2f}")
    print(f"   - Taxes: ${user_coverage_breakdown['taxes']:,.2f}")
    print(f"   - Net Benefits: ${user_coverage_breakdown['net_benefits']:,.2f}")
else:
    print("   - No existing coverage")
print()
print(f"3. Remaining Coverage Gap: ${coverage_gap:,.2f}")
if coverage_gap > 0:
    print(f"   - Recommended Coverage: {benefit_percentage:.1f}% of income")
    print(f"   - Duration: {years_until_benefit_cutoff_age:.1f} years (until age {BENEFIT_CUTOFF_AGE})")
    print(f"   - Current Annual Income: ${current_annual_income:,.2f}")
else:
    print("   - No additional coverage needed")
print()
print("=" * 70)

## Partner Disability Insurance Calculation

Calculate disability insurance needs for partner (if partner exists).

In [ ]:
# Check if partner exists and has income profiles
if user_config.partner and user_config.partner.income_profiles and len(user_config.partner.income_profiles) > 0:
    print("Partner found with income profiles. Calculating partner disability insurance needs...")
    
    # Restore original config and zero out partner job income for partner disability scenario
    # Reload engine with original config to restore user income
    engine_restored = SimulationEngine(config_path=config_path, trial_qty=1)
    set_fixed_inflation(engine_restored, 0.02)  # Set 2% annual inflation for deterministic analysis
    
    # Zero out partner job income (keep user income)
    if engine_restored._user_config.partner and engine_restored._user_config.partner.income_profiles:
        engine_restored._user_config.partner.income_profiles = []
    
    # Run partner disability scenario
    try:
        engine_restored.gen_all_trials()
        
        partner_disability_results = engine_restored.results
        partner_disability_dfs = partner_disability_results.as_dataframes()
        assert len(partner_disability_dfs) == 1, "Expected exactly one trial DataFrame"
        partner_disability_df = partner_disability_dfs[0]
        partner_disability = RealFinancialData(partner_disability_df)
    except Exception as e:
        print(f"ERROR: Failed to run partner disability scenario: {e}")
        print("Please check your configuration file and ensure all required fields are present.")
        raise
    
    
    
    # Calculate partner income differences
    # SUM of lost partner job income + reduced SS + reduced pension (per FR-006 and FR-008)
    # Per FR-011 and FR-012: Verify that SimulationEngine properly reflects Social Security and pension
    # benefit differences when partner job income is zeroed out (baseline vs disability scenarios)
    if user_config.partner.age is None:
        raise Exception("Partner age is not set")
    partner_benefit_cutoff_age_date = BENEFIT_CUTOFF_AGE - user_config.partner.age + TODAY_YR_QT
    partner_mask_until_benefit_cutoff_age = partner_disability.dates <= partner_benefit_cutoff_age_date
    
    # Calculate partner total replacement needs using post-tax income
    # Per FR-006 and FR-008: Partner total replacement needs = Baseline post-tax income - Partner disability post-tax income
    # where post-tax income = (Job Income + Social Security + Pension) - (Income Taxes + Medicare Taxes)
    # Note: baseline_post_tax_real_q already calculated above for user scenario
    partner_total_replacement_needs = baseline.post_tax_total_lifetime - partner_disability.post_tax_total_lifetime
    
    # Also calculate component breakdown for display (pre-tax for reference)
    partner_lost_pre_tax_job_income = baseline.pre_tax_job_total_lifetime - partner_disability.pre_tax_job_total_lifetime
    partner_reduced_pre_tax_ss = baseline.pre_tax_ss_total_lifetime - partner_disability.pre_tax_ss_total_lifetime
    partner_reduced_pre_tax_pension = baseline.pre_tax_pension_total_lifetime - partner_disability.pre_tax_pension_total_lifetime
    
    # Total replacement needs is the SUM of all components (per FR-006 and FR-008)
    
    # Calculate partner existing coverage replacement
    partner_coverage = disability_coverage["partner_disability_coverage"]
    partner_coverage_pct = partner_coverage.get("percentage", 0.0)
    partner_coverage_duration = partner_coverage.get("duration_years", 0)
    
    if partner_coverage_pct > 0 and partner_coverage_duration > 0:
        partner_existing_coverage, partner_coverage_breakdown = calculate_post_tax_existing_coverage(
            baseline_df, partner_coverage_pct, partner_coverage_duration
        )
    else:
        partner_existing_coverage = 0.0
        partner_coverage_breakdown = {"gross_benefits": 0.0, "taxes": 0.0, "net_benefits": 0.0}
    
    # Calculate partner coverage gap
    partner_coverage_gap = max(0, partner_total_replacement_needs - partner_existing_coverage)
    
    # Calculate partner benefit percentage
# Formula per FR-008: (Coverage gap / Years until Benefit cutoff age) / Current annual income
    partner_years_until_benefit_cutoff_age = calculate_years_until_benefit_cutoff_age(user_config.partner.age)
    partner_current_annual_income = get_current_annual_income(user_config, is_partner=True)
    
    if partner_years_until_benefit_cutoff_age > 0 and partner_current_annual_income > 0:
        partner_benefit_percentage = (partner_coverage_gap / partner_years_until_benefit_cutoff_age) / partner_current_annual_income * 100
    else:
        partner_benefit_percentage = 0.0
    
    # Display partner results
    print("=" * 70)
    print("PARTNER DISABILITY INSURANCE CALCULATION RESULTS")
    print("=" * 70)
    print()
    print(f"1. Total Income Replacement Needed: ${partner_total_replacement_needs:,.2f}")
    print(f"   - Lost Job Income (until Benefit cutoff age): ${partner_lost_pre_tax_job_income:,.2f}")
    print(f"   - Reduced Social Security (lifetime, including post-Benefit cutoff age): ${partner_reduced_pre_tax_ss:,.2f}")
    print(f"   - Reduced Pension (lifetime, including post-Benefit cutoff age): ${partner_reduced_pre_tax_pension:,.2f}")
    print()
    print(f"2. Existing Coverage Replacement (after taxes): ${partner_existing_coverage:,.2f}")
    if partner_coverage_pct > 0:
        print(f"   - Coverage: {partner_coverage_pct}% of income for {partner_coverage_duration} years")
        print(f"   - Gross Benefits: ${partner_coverage_breakdown['gross_benefits']:,.2f}")
        print(f"   - Taxes: ${partner_coverage_breakdown['taxes']:,.2f}")
        print(f"   - Net Benefits: ${partner_coverage_breakdown['net_benefits']:,.2f}")
    else:
        print("   - No existing coverage")
    print()
    print(f"3. Remaining Coverage Gap: ${partner_coverage_gap:,.2f}")
    if partner_coverage_gap > 0:
        print(f"   - Recommended Coverage: {partner_benefit_percentage:.1f}% of income")
        print(f"   - Duration: {partner_years_until_benefit_cutoff_age:.1f} years (until Benefit cutoff age)")
        print(f"   - Current Annual Income: ${partner_current_annual_income:,.2f}")
    else:
        print("   - No additional coverage needed")
    print()
    print("=" * 70)
else:
    print("No partner with income profiles found. Skipping partner calculation.")